In [1]:
# Google Colab-only setup. No need to run this cell in other environments.
 
# Mount my Google Drive root folder
from google.colab import drive
drive.mount('/content/drive')

# cd to bayesian-dl-experiments directory
%cd 'drive/My Drive/Colab Notebooks/bayesian-dl-experiments'
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/bayesian-dl-experiments
datasets_files	experiments.ipynb  LICENSE  README.md  ronald_bdl


## Experiment Setup

In [0]:
import torch
import numpy as np

# IPython reloading magic
%load_ext autoreload
%autoreload 2

# Random seeds
# Based on https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(682)
np.random.seed(682)

# torch.device / CUDA Setup
use_cuda = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    torch_device = torch.device('cpu')

# Dataset to use
dataset_name = 'yacht'

# Training set size
dataset_train_size = 0.8

# Epochs
n_epochs = 10

## Prepare data

### Get the data as a torch Dataset object

In [3]:
from ronald_bdl import datasets

if dataset_name == 'MNIST':
    dataset = datasets.MNIST(root_dir='./datasets_files', transform=None, download=True)
else:
    dataset = datasets.UCIDatasets(dataset_name, root_dir='./datasets_files', transform=None, download=True)

# Print the size of the dataset
print("dataset size = " + str(dataset.data.shape))

Using downloaded and verified file: ./datasets_files/yacht/yacht_hydrodynamics.data
dataset sizetorch.Size([308, 7])


### Split into training/test set using torch's DataLoader

In [0]:
from torch.utils.data import random_split, DataLoader

train_size = int(dataset_train_size * len(dataset))
test_size = len(dataset) - train_size

train, test = random_split(dataset, lengths=[train_size, test_size])

train_loader = DataLoader(train, batch_size=10)
test_loader = DataLoader(test, batch_size=1)

## Define network

In [5]:
from ronald_bdl import models

network = models.FCNetMCDropout(
    input_dim=len(dataset.features), 
    output_dim=len(dataset.targets),
    hidden_dim=100,
    n_hidden=2,
    n_predictions=10000,
    dropout_rate=0.01,
)

# Send the whole model to the selected torch.device
network.to(torch_device)

# Print the network structure
print(network)

FCNetMCDropout(
  (input): Linear(in_features=6, out_features=100, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
  )
  (output): Linear(in_features=100, out_features=1, bias=True)
)


## Train the network

### Setup optimizer

In [0]:
from torch import nn, optim

objective = nn.MSELoss()
optimizer = optim.Adam(network.parameters(), lr=0.01)

### Run the optimizer

In [7]:
# Partially adapted from https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
for epoch in range(n_epochs): # loop over the dataset multiple times

    print("Running epoch " + str(epoch))

    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, targets = data

        # Store the batch to torch_device's memory
        inputs = inputs.to(torch_device)
        targets = targets.to(torch_device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = network(inputs)

        loss = objective(outputs, targets)
        loss.backward()

        optimizer.step()

Running epoch 0
Running epoch 1
Running epoch 2
Running epoch 3
Running epoch 4
Running epoch 5
Running epoch 6
Running epoch 7
Running epoch 8
Running epoch 9


## Make predictions

In [8]:
for i, data in enumerate(test_loader):
    # Get the test data
    inputs, targets = data

    # Store the batch to torch_device's memory
    inputs = inputs.to(torch_device)
    targets = targets.to(torch_device)
    
    predictions, mean, var = network.mc_predict(inputs)

    print("Test " + str(i))
    print("Mean = " + str(mean))
    print("Variance = " + str(var))
    print()

Test 0
Mean = tensor([5.8063], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.4840], device='cuda:0', grad_fn=<VarBackward1>)

Test 1
Mean = tensor([2.8014], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.2735], device='cuda:0', grad_fn=<VarBackward1>)

Test 2
Mean = tensor([0.9864], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.2005], device='cuda:0', grad_fn=<VarBackward1>)

Test 3
Mean = tensor([13.1073], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.8315], device='cuda:0', grad_fn=<VarBackward1>)

Test 4
Mean = tensor([2.1171], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.3777], device='cuda:0', grad_fn=<VarBackward1>)

Test 5
Mean = tensor([0.5097], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.0019], device='cuda:0', grad_fn=<VarBackward1>)

Test 6
Mean = tensor([8.9506], device='cuda:0', grad_fn=<MeanBackward1>)
Variance = tensor([0.6228], device='cuda:0', grad_fn=<VarBackward1>)